<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Features-to-add" data-toc-modified-id="Features-to-add-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Features to add</a></span></li></ul></div>

Here I am merging tables onto the main table, transfers.

## Features to add

Adding for sure:

- Prescriptions, which is ready

- Admissions: check ppoint, non-white?, marital status??, medicaid, language?

- Patients: Age and gender

- POE: Many categories (order type), but decent. 


Maybe adding in the future:

- Procedures by ICD code, could be interesting but lots of features

- Lab events: A LOT of features, although you can just add flag (associated with storetime) and priority (associated with charttime of test (frequency encoding?)

- OMR: Blood pressure, weight... Interesting but probably there will be many missing values and will be hard to incorporate into dataframe

- hcpcsevents: A LOT of features

- Microbiology events: A LOT of features (no outcomes clear, although there are comments of test)


<br><br>

Some questions: 

Should I increase the granularity of the prescriptions (go down to level 2, 3 or 4). That would add between 1K and 5K features

Can I leave duration of transfer in datetime or should I categorise it/bin it?
bin

Should I bin age or can be quantitative?
Bin it (maybe check before when it's quantitative)

Is it okay adding POE? Should I stop there or add also the other tables? It would mean lots of features. 

Should I add measures such as blood pressure and weight even if values will be missing in many rows? And again do I have to bin?
Leave it for now and maybe with the PCA it tells you something. Maybe you see that could be good to include

Focus on not adding any extra features but focus on step 1 (do PCA, fit models and decide whether to add more features or not)

Step 1
For small dataset
logistic regression SVM, decision tree and random forest, naive bayes
Group the classes before fitting the model
PCA to see if I can compress the model as it is right now
See if by going up on the number of features you are getting any gain
How many orders for now and that's it

Step 2
For large dataset

For dataset try to make bins if possible, 5-10

Distribution is skewed. But after we have converted (the class) into binary, the problem we have is bias (the model is biased).

In [3]:
import numpy as np
import pandas as pd
import copy
import nachopy
import importlib
import pickle

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats = ['svg'] # Setting plots as vector images

plt.rcParams['figure.figsize'] = (10, 5)
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import matplotlib as mpl
import seaborn as sns

In [4]:
importlib.reload(nachopy);

In [5]:
transfers = pd.read_csv('../Capstone data/MIMIC IV/transfers.csv')

In [6]:
transfers.head()

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10000032,22595853.0,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00
1,10000032,22595853.0,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27
2,10000032,22595853.0,36904543,discharge,NaN,2180-05-07 17:21:27,NaN
3,10000032,22841357.0,34100253,discharge,NaN,2180-06-27 18:49:12,NaN
4,10000032,22841357.0,34703856,admit,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12


In [8]:
transfers['hadm_id'] = transfers.hadm_id.astype('Int64')

In [ ]:
display(transfers.head())

In [9]:
transfers.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991704 entries, 0 to 1991703
Data columns (total 7 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   subject_id   1991704 non-null  int64 
 1   hadm_id      1644130 non-null  Int64 
 2   transfer_id  1991704 non-null  int64 
 3   eventtype    1991704 non-null  object
 4   careunit     1537380 non-null  object
 5   intime       1991704 non-null  object
 6   outtime      1537383 non-null  object
dtypes: Int64(1), int64(2), object(4)
memory usage: 108.3+ MB


In [11]:
transfers['intime'] = pd.to_datetime(transfers.intime, infer_datetime_format=True)
transfers['outtime'] = pd.to_datetime(transfers.outtime, infer_datetime_format=True)

In [12]:
transfers.nunique()

subject_id      315460
hadm_id         454324
transfer_id    1991704
eventtype            4
careunit            39
intime         1980736
outtime        1532471
dtype: int64

In [13]:
transfers_dupl = transfers.drop(columns = 'transfer_id').duplicated()
print(transfers_dupl.sum())
transfers = transfers[~transfers_dupl]

8


In [14]:
transfers[transfers.hadm_id == 22595853]

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10000032,22595853,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00
1,10000032,22595853,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27
2,10000032,22595853,36904543,discharge,NaN,2180-05-07 17:21:27,NaT


In [15]:
transfers.sort_values(by = ['subject_id','intime'])

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10000032,22595853,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00
1,10000032,22595853,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27
2,10000032,22595853,36904543,discharge,NaN,2180-05-07 17:21:27,NaT
5,10000032,22841357,38112554,ED,Emergency Department,2180-06-26 15:54:00,2180-06-26 21:31:00
4,10000032,22841357,34703856,admit,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12
...,...,...,...,...,...,...,...
1991699,19999914,<NA>,32002659,ED,Emergency Department,2158-12-24 11:41:00,2158-12-24 11:56:00
1991701,19999987,23865745,34731548,ED,Emergency Department,2145-11-02 19:28:00,2145-11-02 22:59:00
1991702,19999987,23865745,36195440,admit,Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30
1991700,19999987,23865745,30249304,transfer,Neurology,2145-11-04 21:29:30,2145-11-11 13:00:47


Example of an stay with three transfer events. The patient is admitted in the emergency department (event type ED, careunit Emergency Department) and stays there for 4 hours. He is then transferred to the transplant careunit, for which he needs to be admitted into hospital (eventtype admit). After 1 day he is discharged. We do not need the discharge rows, as there is nothing to be predicted from those. Rather, discharge should be the "output" for row 2. Likewise, the output for row 1 will not be ED or Emergency Department (that is a feature), but Transplant. Discharge have no duration and the resulting careunits and outtime are null values, although that is expected

In [16]:
print(transfers[transfers.outtime.isna()].nunique())

subject_id     190279
hadm_id        454321
transfer_id    454321
eventtype           1
careunit            0
intime         454236
outtime             0
dtype: int64


In [17]:
transfers.sort_values(by=['subject_id','intime'],inplace = True)
events = transfers.careunit.loc[1:].values # From second row, getting the destination careunit (class)
transfers = transfers.iloc[:-1] # Until second to last row
transfers['event'] = events

C:\Users\Nacho\AppData\Local\Temp\ipykernel_12244\1795162515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transfers.sort_values(by=['subject_id','intime'],inplace = True)


Now there should be one more nan values in the new event column than in the discharge column

In [18]:
print(sum(transfers.event.isna()))
print(sum(transfers.eventtype == 'discharge'))

454324
454323


It's correct, so we can fill the NaNs in the event column as discharges and drop the (old) discharge columns

In [19]:
transfers['event'] = transfers.event.fillna('Discharge')
transfers.dropna(axis = 0, subset = 'careunit', inplace = True)
transfers.reset_index(inplace = True,drop = True)

In [20]:
transfers.outtime.isna().sum()

0

Let's start merging tables. We can begin with the patients table, which contains information on age and gender of patient

In [21]:
prescriptions = pd.read_pickle('../Capstone data/pickles/prescriptions_final.pkl')
cat = list('ABCDGHJLMNPRSV')

In [22]:
prescriptions.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17628928 entries, 0 to 17628927
Data columns (total 17 columns):
 #   Column      Non-Null Count     Dtype 
---  ------      --------------     ----- 
 0   subject_id  17628928 non-null  int64 
 1   hadm_id     17628928 non-null  int64 
 2   starttime   17621552 non-null  object
 3   A           17628928 non-null  int8  
 4   B           17628928 non-null  int8  
 5   C           17628928 non-null  int8  
 6   D           17628928 non-null  int8  
 7   G           17628928 non-null  int8  
 8   H           17628928 non-null  int8  
 9   J           17628928 non-null  int8  
 10  L           17628928 non-null  int8  
 11  M           17628928 non-null  int8  
 12  N           17628928 non-null  int8  
 13  P           17628928 non-null  int8  
 14  R           17628928 non-null  int8  
 15  S           17628928 non-null  int8  
 16  V           17628928 non-null  int8  
dtypes: int64(2), int8(14), object(1)
memory usage: 773.4+ MB


No null values woow... But we know it is not true. Nearly 2% of all prescription rows have not been categorised and labelled as -1. Because the V in the categories stands for Varied and it tends to work as a catch-all, we are going to put them into this category. At least this way they we wll be able to count the number of prescriptions per patient. NO VARIOUS CHECK THIS, SETTING COUNTER ON PRES_NUMBER

In [24]:
missing_cats_bool = prescriptions.A<0
prescriptions.loc[missing_cats_bool,cat] = 0
prescriptions['starttime'] = pd.to_datetime(prescriptions.starttime,infer_datetime_format=True)

In [25]:
prescriptions['pres_number'] = 1 # Setting counter of overall prescriptions number

And there are no times in the outtime as we have dropped all the discharge rows.

In [26]:
transfers.reset_index(drop = True, inplace = True)
transfers.sort_values(by = ['subject_id','intime'],inplace = True)

In [27]:
cat.append('pres_number')

In [36]:
transfers = transfers_inner.copy()

In [29]:
def step_merger(dfl,dfr,on_col = 'subject_id',n = 15,dums = cat,index_col = 'transfer_id'):
    
    df_merged = pd.merge(dfl.head(0),dfr.head(0), how = 'left')

    non_dums = list(df_merged.columns.values)
    [non_dums.remove(col) for col in dums]
    dums = copy.deepcopy(dums)
    dums.append(index_col)

    df_merged.drop(columns = index_col,inplace = True)
    df_merged.index.rename(index_col,inplace = True)

    dfl.sort_values(by = on_col,inplace=True)
    dfl.reset_index(inplace = True, drop = True)
    dfr.sort_values(by = on_col,inplace=True)
    dfr.reset_index(inplace = True, drop = True)

    ind_l1 = 0
    ind_r1 = 0

    inc = round(len(dfr)/n)
    max_l = dfl[on_col].max()
    max_r = dfr[on_col].max()
    for i in range(n):
        # Taking first lim rows 
        max_oncol = dfr.loc[ind_r1:].head(inc)[on_col].values[-1]

        if max_oncol < max_l and max_oncol < max_r:
            ind_r2 = (dfr[on_col]> max_oncol).argmax() - 1
            dfr_chunk = dfr.loc[ind_r1:ind_r2]

            ind_l2 = dfl[dfl[on_col] == dfr_chunk[on_col].max()].index.values[-1]

            dfl_chunk = dfl.reset_index(drop = True).loc[ind_l1:ind_l2]

        else:
            dfr_chunk = dfr.loc[ind_r1:]
            dfl_chunk = dfl.loc[ind_l1:]

        df_merging = pd.merge(dfl_chunk,dfr_chunk, on = on_col, how = 'left')

        valid_rows = (df_merging.starttime>df_merging.intime) & (df_merging.starttime<df_merging.outtime)
        df_merging.loc[~valid_rows,dums[:-1]] = 0

        df_merging = pd.concat([df_merging[dums].groupby(index_col).sum().astype(int),
                                df_merging[non_dums].groupby(index_col).first()],
                                axis = 1)
        df_merged = pd.concat([df_merged,df_merging], axis = 0)
        ind_r1 = ind_r2 + 1
        ind_l1 = ind_l2 + 1
        print(f'Loop {i + 1} out of {n}')
    
    df_merged.reset_index(drop = False, inplace = True)
    return df_merged

In [30]:
transfers = step_merger(transfers,prescriptions.rename(columns = {'hadm_id':'hadm_id_pres'}))

Loop 1 out of 15
Loop 2 out of 15
Loop 3 out of 15
Loop 4 out of 15
Loop 5 out of 15
Loop 6 out of 15
Loop 7 out of 15
Loop 8 out of 15
Loop 9 out of 15
Loop 10 out of 15
Loop 11 out of 15
Loop 12 out of 15
Loop 13 out of 15
Loop 14 out of 15
Loop 15 out of 15


In [34]:
transfers_inner

,transfer_id,hadm_id,eventtype,careunit,intime,outtime,event,subject_id,hadm_id_pres,starttime,...,H,J,L,M,N,P,R,S,V,pres_number
0,30000188,27072986,transfer,Neurology,2154-01-29 18:00:34,2154-01-29 20:49:56,Emergency Department Observation,10006457,38900309.0,2153-07-27 18:38:00,...,0,0,0,0,0,0,0,0,0,0
1,30000295,28050343,admit,Discharge Lounge,2161-12-26 01:24:16,2161-12-26 20:36:37,Med/Surg/GYN,10646287,23684941.0,2171-01-23 02:00:00,...,0,1,0,1,0,0,1,2,2,6
2,30000417,<NA>,ED,Emergency Department,2133-09-10 10:34:00,2133-09-10 13:13:00,Surgery,10275184,22066681.0,2132-06-01 00:00:00,...,0,0,1,2,5,0,5,1,0,16
3,30000587,27833084,admit,Hematology/Oncology,2174-10-29 11:01:26,2174-11-01 18:48:29,Discharge,10279453,27015038.0,2174-11-18 13:00:00,...,0,0,0,1,6,0,3,2,0,19
4,30000642,28921361,admit,Hematology/Oncology,2160-09-02 17:36:00,2160-09-02 18:00:02,Hematology/Oncology,10015367,24446768.0,2157-10-12 20:00:00,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1537367,39999449,25579132,ED,Emergency Department,2149-06-26 16:38:00,2149-06-26 23:21:00,Surgery/Pancreatic/Biliary/Bariatric,19831776,29913061.0,2148-11-15 22:00:00,...,0,0,0,0,1,0,0,1,1,4
1537368,39999477,21330056,transfer,PACU,2166-04-29 08:38:40,2166-04-29 10:14:25,Cardiac Vascular Intensive Care Unit (CVICU),19529415,21330056.0,2166-04-24 21:00:00,...,0,0,0,0,0,0,0,0,0,0
1537369,39999495,23212325,transfer,Medicine,2144-04-29 01:20:01,2144-05-05 15:29:34,Discharge,19818766,23212325.0,2144-04-28 14:00:00,...,0,0,0,0,6,0,3,1,7,26
1537370,39999812,28652249,transfer,Medicine,2165-01-30 15:49:51,2165-02-01 01:48:46,Medicine,19978766,21880865.0,2165-03-27 12:00:00,...,0,0,0,0,2,0,0,0,0,3


In [37]:
display(transfers.sort_values(by = ['subject_id', 'intime']))
print(transfers.info(show_counts = True))

,transfer_id,hadm_id,eventtype,careunit,intime,outtime,event,subject_id,hadm_id_pres,starttime,...,H,J,L,M,N,P,R,S,V,pres_number
33714,33258284,22595853,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00,Transplant,10000032,22595853.0,2180-05-07 01:00:00,...,0,2,0,0,1,0,2,0,0,9
53947,35223874,22595853,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27,Discharge,10000032,22595853.0,2180-05-07 01:00:00,...,0,3,0,0,2,0,2,1,0,12
83714,38112554,22841357,ED,Emergency Department,2180-06-26 15:54:00,2180-06-26 21:31:00,Transplant,10000032,22595853.0,2180-05-07 01:00:00,...,0,3,0,0,1,0,1,0,0,12
48613,34703856,22841357,admit,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12,Discharge,10000032,22595853.0,2180-05-07 01:00:00,...,0,4,0,0,1,0,3,1,0,15
30386,32952584,29079034,ED,Emergency Department,2180-07-22 16:24:00,2180-07-23 05:54:00,Emergency Department,10000032,22595853.0,2180-05-07 01:00:00,...,0,3,0,0,2,0,2,0,0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484027,34767778,21033226,transfer,Surgical Intensive Care Unit (SICU),2164-09-13 09:37:48,2164-09-17 16:35:15,Discharge,19999840,21033226.0,2164-09-16 16:00:00,...,0,8,0,0,9,0,0,7,12,63
1455757,32002659,<NA>,ED,Emergency Department,2158-12-24 11:41:00,2158-12-24 11:56:00,Emergency Department,19999914,NaN,NaT,...,0,0,0,0,0,0,0,0,0,0
1483612,34731548,23865745,ED,Emergency Department,2145-11-02 19:28:00,2145-11-02 22:59:00,Trauma SICU (TSICU),19999987,23865745.0,2145-11-04 10:00:00,...,0,0,0,0,0,0,0,0,0,0
1498411,36195440,23865745,admit,Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,Neurology,19999987,23865745.0,2145-11-04 10:00:00,...,4,1,0,0,11,0,3,4,4,37


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1537372 entries, 0 to 1537371
Data columns (total 25 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   transfer_id   1537372 non-null  int64         
 1   hadm_id       1189806 non-null  Int64         
 2   eventtype     1537372 non-null  object        
 3   careunit      1537372 non-null  object        
 4   intime        1537372 non-null  datetime64[ns]
 5   outtime       1537372 non-null  datetime64[ns]
 6   event         1537372 non-null  object        
 7   subject_id    1537372 non-null  int64         
 8   hadm_id_pres  1410027 non-null  float64       
 9   starttime     1410024 non-null  datetime64[ns]
 10  A             1537372 non-null  int32         
 11  B             1537372 non-null  int32         
 12  C             1537372 non-null  int32         
 13  D             1537372 non-null  int32         
 14  G             1537372 non-null  int32         
 15

In [38]:
transfers.drop(columns = ['starttime'],inplace = True)

In [39]:
patients = pd.read_csv('../Capstone data/MIMIC IV/patients.csv')
patients.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,NaN


In [40]:
patients['dod'] = pd.to_datetime(patients.dod, infer_datetime_format = True)

Basic eda:

In [41]:
print(patients.info(show_counts = True))
print('\n')
print(patients.dod.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315460 entries, 0 to 315459
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   subject_id         315460 non-null  int64         
 1   gender             315460 non-null  object        
 2   anchor_age         315460 non-null  int64         
 3   anchor_year        315460 non-null  int64         
 4   anchor_year_group  315460 non-null  object        
 5   dod                30636 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 14.4+ MB
None


count                   30636
unique                  19705
top       2129-11-19 00:00:00
freq                        7
first     2104-12-24 00:00:00
last      2212-01-22 00:00:00
Name: dod, dtype: object


Okay, no null values (other that the patients that did not die) and nothing weird on date of death (dod). Let's look at info on gender and age.

In [42]:
pd.Series(patients.gender.isna().mean()*100)

0    0.0
dtype: float64

In [43]:
nachopy.display_sbs(
                [patients.gender.value_counts(normalize = True)*100,\

                patients.loc[~patients.dod.isna(),'gender'].value_counts()/patients.gender.value_counts()*100,\

                pd.concat([patients.anchor_age[patients.gender == 'M'].describe().rename('Males'),\
                patients.anchor_age[patients.gender == 'F'].describe().rename('Females')], axis = 1)],\

                titles = ['Patient genders', '% that died', 'Male and female age distributions']
)

Patient genders 
 
 
   
 gender 
 
 
 
 
 F 
 52.906549 
 
 
 M 
 47.093451 
 
 
 

 
 % that died 
 
 
   
 gender 
 
 
 
 
 F 
 8.775367 
 
 
 M 
 10.763255 
 
 
 

 
 Male and female age distributions 
 
 
   
 Males 
 Females 
 
 
 
 
 count 
 148561.000000 
 166899.000000 
 
 
 mean 
 49.026588 
 48.080480 
 
 
 std 
 20.146493 
 21.515040 
 
 
 min 
 18.000000 
 18.000000 
 
 
 25% 
 30.000000 
 28.000000 
 
 
 50% 
 49.000000 
 46.000000 
 
 
 75% 
 65.000000 
 65.000000 
 
 
 max 
 91.000000 
 91.000000

Okay, good for now. We do not have any death registered in our transfers tables. This should be an event different from discharge, as the features for patients that expired and patients that were discharged from the hospital will probably be significantly different. Now we are going to join gender, age and dod.

In [44]:
transfers = pd.merge(transfers,patients[['subject_id','gender','anchor_age','dod']], on = 'subject_id',how = 'left')
transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537372 entries, 0 to 1537371
Data columns (total 27 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   transfer_id   1537372 non-null  int64         
 1   hadm_id       1189806 non-null  Int64         
 2   eventtype     1537372 non-null  object        
 3   careunit      1537372 non-null  object        
 4   intime        1537372 non-null  datetime64[ns]
 5   outtime       1537372 non-null  datetime64[ns]
 6   event         1537372 non-null  object        
 7   subject_id    1537372 non-null  int64         
 8   hadm_id_pres  1410027 non-null  float64       
 9   A             1537372 non-null  int32         
 10  B             1537372 non-null  int32         
 11  C             1537372 non-null  int32         
 12  D             1537372 non-null  int32         
 13  G             1537372 non-null  int32         
 14  H             1537372 non-null  int32         
 15

In [45]:
# deceased = transfers.dod == transfers.outtime.dt.date
# print('Did any patients die at event "Discharge"?',deceased.any())
# print('How many?', deceased.sum(),'And the percentage?', round(deceased.mean()*100,2))
# transfers[deceased].head()
# transfers[['gender','anchor_age']].isna().mean()
# transfers.loc[deceased,'event'] = 'Deceased'
# transfers.drop(columns = 'dod',inplace = True)
# print('NaN values for new columns')
# print(transfers[['gender','anchor_age']].isna().mean())

Less than 1% of patients in the transfers column died rather than get discharged. We will change the class to "Deceased" for these rows. We can drop the dod column and check for any missing values in the new columns. 

Okay, clean df again. Now let's check what we can add from admissions (private info not shown)

In [65]:
admissions = pd.read_csv('../Capstone data/MIMIC IV/admissions.csv')
# display(admissions.head())
print(admissions.info(show_counts = True))
print(admissions.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454324 entries, 0 to 454323
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   subject_id            454324 non-null  int64 
 1   hadm_id               454324 non-null  int64 
 2   admittime             454324 non-null  object
 3   dischtime             454324 non-null  object
 4   deathtime             9087 non-null    object
 5   admission_type        454324 non-null  object
 6   admission_location    454324 non-null  object
 7   discharge_location    328519 non-null  object
 8   insurance             454324 non-null  object
 9   language              454324 non-null  object
 10  marital_status        444608 non-null  object
 11  race                  454324 non-null  object
 12  edregtime             315460 non-null  object
 13  edouttime             315460 non-null  object
 14  hospital_expire_flag  454324 non-null  int64 
dtypes: int64(3), obje

Okay, let's do a basic EDA of some variables that could be interesting

In [47]:
columns = ['admission_type','discharge_location','insurance','language','marital_status','race']
nachopy.display_sbs([admissions[col].value_counts() for col in columns],max_rows = 17, suffix = 'value_counts')

,admission_type
EW EMER.,157487
EU OBSERVATION,100133
OBSERVATION ADMIT,55210
URGENT,47127
SURGICAL SAME DAY ADMISSION,35994
DIRECT EMER.,20584
DIRECT OBSERVATION,19698
ELECTIVE,11089
AMBULATORY OBSERVATION,7002
,discharge_location


Admission type seems quite relevant, we could look into doing some label encoding. Discharge location also looks relevant and should probably be used to update the class ("event"). Insurance relevant as well, as medicaid and medicare patients could have different outcomes than privately-insured. English and marital status will be added for now, although probably little relevance. Race may be relevant, could be added simply as a dummy column (1 if white, 0 otherwise). We will join now and adjust later during the EDA. 

In [48]:
transfers['hadm_id_nans'] = transfers.hadm_id
transfers.loc[transfers.hadm_id.isna(),'hadm_id'] =  transfers.index[transfers.hadm_id.isna()] # Just to try and fill most of them
columns.append('hadm_id')
transfers = pd.merge(transfers,admissions[columns], how = 'left', on = 'hadm_id')
transfers.drop(columns = 'hadm_id',inplace = True)
transfers.rename(columns = {'hadm_id_nans':'hadm_id'},inplace = True)
transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537372 entries, 0 to 1537371
Data columns (total 33 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   transfer_id         1537372 non-null  int64         
 1   eventtype           1537372 non-null  object        
 2   careunit            1537372 non-null  object        
 3   intime              1537372 non-null  datetime64[ns]
 4   outtime             1537372 non-null  datetime64[ns]
 5   event               1537372 non-null  object        
 6   subject_id          1537372 non-null  int64         
 7   hadm_id_pres        1410027 non-null  float64       
 8   A                   1537372 non-null  int32         
 9   B                   1537372 non-null  int32         
 10  C                   1537372 non-null  int32         
 11  D                   1537372 non-null  int32         
 12  G                   1537372 non-null  int32         
 13  H           

First, let's see if there are any subject_id that has a hadm_id on the admissions table but not on the transfers table. For race insurance and language there are no missing values on the admissions table, so we can try to retrieve them by updating using subject_id for the first occurence of these values

In [49]:
columns.append('subject_id')
missing_rows = admissions[columns].groupby('subject_id').first()

transfers.reset_index(drop = False)
transfers.set_index('subject_id',drop = False,inplace = True)
transfers.update(missing_rows)
transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537372 entries, 10006457 to 19840299
Data columns (total 33 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   transfer_id         1537372 non-null  int64         
 1   eventtype           1537372 non-null  object        
 2   careunit            1537372 non-null  object        
 3   intime              1537372 non-null  datetime64[ns]
 4   outtime             1537372 non-null  datetime64[ns]
 5   event               1537372 non-null  object        
 6   subject_id          1537372 non-null  int64         
 7   hadm_id_pres        1410027 non-null  float64       
 8   A                   1537372 non-null  int32         
 9   B                   1537372 non-null  int32         
 10  C                   1537372 non-null  int32         
 11  D                   1537372 non-null  int32         
 12  G                   1537372 non-null  int32         
 13  H   

We have increased our number of hadm_ids and also the other new columns, which is good. We could now try to fill the missing values using values from the table itself.

In [50]:
missing_rows = transfers.drop(columns = ['subject_id','dod','discharge_location']).groupby('subject_id').first()
transfers.update(missing_rows)
transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537372 entries, 10006457 to 19840299
Data columns (total 33 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   transfer_id         1537372 non-null  int64         
 1   eventtype           1537372 non-null  object        
 2   careunit            1537372 non-null  object        
 3   intime              1537372 non-null  datetime64[ns]
 4   outtime             1537372 non-null  datetime64[ns]
 5   event               1537372 non-null  object        
 6   subject_id          1537372 non-null  int64         
 7   hadm_id_pres        1410027 non-null  float64       
 8   A                   1537372 non-null  int32         
 9   B                   1537372 non-null  int32         
 10  C                   1537372 non-null  int32         
 11  D                   1537372 non-null  int32         
 12  G                   1537372 non-null  int32         
 13  H   

No luck, we have only filled a few marital_status values (which are the most liable to change from the columns). We can maybe fill some features using the edstays from ED, let's try that

In [51]:
edstays = pd.read_csv('../Capstone data/MIMIC IV/ed/edstays.csv')
edstays.head()

,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition
0,10000032,22595853.0,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,F,WHITE,AMBULANCE,ADMITTED
1,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,F,WHITE,AMBULANCE,ADMITTED
2,10000032,25742920.0,35968195,2180-08-05 20:58:00,2180-08-06 01:44:00,F,WHITE,AMBULANCE,ADMITTED
3,10000032,29079034.0,32952584,2180-07-22 16:24:00,2180-07-23 05:54:00,F,WHITE,AMBULANCE,HOME
4,10000032,29079034.0,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,F,WHITE,AMBULANCE,ADMITTED


The ED collects different data than the hospital, which is unfortunate. We will leave these features as they are for now.carry out analysis regardless or maybe we decide to exclude ED stays in the future. It appears we can add race (we can just group by subject_id and then update the column with the first occurence.

In [52]:
missing_rows = edstays[['hadm_id','race','subject_id']].groupby('subject_id').first()
transfers.update(missing_rows)
transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537372 entries, 10006457 to 19840299
Data columns (total 33 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   transfer_id         1537372 non-null  int64         
 1   eventtype           1537372 non-null  object        
 2   careunit            1537372 non-null  object        
 3   intime              1537372 non-null  datetime64[ns]
 4   outtime             1537372 non-null  datetime64[ns]
 5   event               1537372 non-null  object        
 6   subject_id          1537372 non-null  int64         
 7   hadm_id_pres        1410027 non-null  float64       
 8   A                   1537372 non-null  int32         
 9   B                   1537372 non-null  int32         
 10  C                   1537372 non-null  int32         
 11  D                   1537372 non-null  int32         
 12  G                   1537372 non-null  int32         
 13  H   

We can fill hadm_id with the hadm_id_pres column (will not get very far but anyway). We can leave stay_id_ind as 1 for these columns that we know are stay_ids.

In [58]:
transfers[cat].describe()

,A,B,C,D,G,H,J,L,M,N,P,R,S,V,pres_number
count,1.537372e+06,1.537372e+06,1.537372e+06,1.537372e+06,1.537372e+06,1.537372e+06,1.537372e+06,1.537372e+06,1.537372e+06,1.537372e+06,1.537372e+06,1.537372e+06,1.537372e+06,1.537372e+06,1.537372e+06
mean,4.254095e+00,3.619635e+00,2.052649e+00,9.704619e-01,5.103963e-01,3.114002e-01,7.570913e-01,1.075875e-01,1.659924e-01,2.250963e+00,5.071837e-02,8.105696e-01,1.167155e+00,1.286221e+00,1.118298e+01
std,8.046271e+00,8.309313e+00,4.147939e+00,2.326693e+00,1.125446e+00,8.814744e-01,1.997583e+00,8.244004e-01,5.306308e-01,4.141005e+00,2.780797e-01,1.816179e+00,2.754683e+00,3.704031e+00,1.901659e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00
75%,5.000000e+00,4.000000e+00,3.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.500000e+01
max,3.860000e+02,6.050000e+02,1.820000e+02,9.600000e+01,3.300000e+01,3.700000e+01,8.100000e+01,8.200000e+01,2.500000e+01,2.160000e+02,1.100000e+01,4.900000e+01,1.400000e+02,2.490000e+02,1.109000e+03


In [59]:
transfers[cat] = transfers[cat].astype('uint16')
transfers['anchor_age'] = transfers.anchor_age.astype('uint8')

In [54]:
transfers['hadm_id'] = transfers.hadm_id.fillna(transfers.hadm_id_pres)
transfers.drop(columns = 'hadm_id_pres',inplace = True)
transfers['hadm_id'] = transfers.hadm_id.astype('Int64')

In [63]:
transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537372 entries, 10006457 to 19840299
Data columns (total 32 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   transfer_id         1537372 non-null  int64         
 1   eventtype           1537372 non-null  object        
 2   careunit            1537372 non-null  object        
 3   intime              1537372 non-null  datetime64[ns]
 4   outtime             1537372 non-null  datetime64[ns]
 5   event               1537372 non-null  object        
 6   subject_id          1537372 non-null  int64         
 7   A                   1537372 non-null  uint16        
 8   B                   1537372 non-null  uint16        
 9   C                   1537372 non-null  uint16        
 10  D                   1537372 non-null  uint16        
 11  G                   1537372 non-null  uint16        
 12  H                   1537372 non-null  uint16        
 13  J   

In [64]:
transfers.to_pickle('../Capstone data/pickles/transfers_v1.pkl')

We filled around half of the remaining missing values for race. Now let's merge with prescriptions

Dropping rows with unknown

In [ ]:
fig1 = plt.figure(figsize = (8,8))
presperevent.sort_values().plot.barh()
plt.title('Patients that ended up in oncology, cardiology and ICU-related\ndepartments had many more prescriptions in previous transfer',pad = 15)
plt.xlabel('Number of prescriptions per transfer')
fig1.savefig('prespertransfervsunit.svg',bbox_inches='tight')

In [ ]:
fig2 = plt.figure(figsize = (10,8))
sns.heatmap(catsperevent.divide(presperevent, axis = 0)*100,cmap='viridis',xticklabels= True)
plt.title('Patients that ended up in different departments were prescribed\ndifferent types of drugs',pad = 15)
plt.xlabel('Number of prescriptions per transfer')
fig2.savefig('drugtypesvcareunit.svg',bbox_inches='tight')

In [ ]:
cat = list('ABCDGHJLMNPRSV')
cat.append('event')
catsperevent = transfers[cat].groupby('event')
catsperevent = catsperevent.sum()/catsperevent.count().sort_index()
presperevent = catsperevent.sum(axis = 1).sort_index(ascending = False)

fig,_ = plt.subplots(2,1,figsize = (8,15))

plt.subplot(2,1,1)
presperevent.sort_values().plot.barh()

plt.subplot(2,1,2)
sns.heatmap(catsperevent.divide(presperevent, axis = 0)*100,cmap='viridis')

plt.show()


In [ ]:
cat = list('ABCDGHJLMNPRSV')
cat.append('event')
catsperevent = transfers[cat].groupby('event')
catsperevent = catsperevent.sum()/catsperevent.count()

Okay, heatmap probably relevant, but I need to normalise the number of prescriptions per event by the duration of the stay

In [ ]:
tran

In [ ]:
poe = pd.read_csv('../Capstone data/MIMIC IV/poe.csv')

In [ ]:
transfers[cat]

In [ ]:
poe.order_type.value_counts(normalize = True)


In [ ]:
cap_utils.display_sbs(['Order subtype value counts'],poe.order_subtype.value_counts(normalize = True))

Probably dropping medications? Could add order sub type and filling the empty values (loads) with the simple order type?

In [ ]:
ed_dupl = edstays.drop(columns = 'stay_id').duplicated()
print(ed_dupl.sum())
edstays = edstays[~ed_dupl]

In [ ]:
edstays.info()

In [ ]:
uniqueid_set = set(transfers.hadm_id.unique())
hadm_len = len(uniqueid_set)
unique_stayid = edstays.stay_id.unique()
stay_len = len(unique_stayid)
set_len = 0
i = 0
while (len(uniqueid_set) > set_len) & (i<stay_len):
    set_len = len(uniqueid_set)
    uniqueid_set.add(unique_stayid[i])
    i +=1
if set_len == hadm_len + stay_len-1:
    print('All good')
else:
    print('caca')

In [ ]:
edstays = edstays[['subject_id','stay_id','intime','outtime']]

In [ ]:
edstays.info()

In [ ]:
edstays['intime'] = pd.to_datetime(edstays.intime,infer_datetime_format=True)
edstays['outtime'] = pd.to_datetime(edstays.outtime,infer_datetime_format=True)

In [ ]:
ed_dupl = edstays.drop(columns = 'stay_id').duplicated()
print(ed_dupl.sum())
edstays = edstays[~ed_dupl]

In [ ]:
transfers = transfers.merge(
                        edstays,
                        left_on = ['subject_id','intime','outtime'], 
                        right_on = ['subject_id','intime','outtime'],
                        how = 'left',
                        copy = False)

In [ ]:
transfers.loc[transfers.hadm_id.isna(),'hadm_id'] = transfers.loc[transfers.hadm_id.isna(),'stay_id']
sum(transfers.hadm_id.isna())

In [ ]:
transfers.careunit.value_counts()

In [ ]:
cat_pres.A.value_counts(normalize = True)*100

In [ ]:
unclass_drugs = cat_pres.drug[cat_pres.A<0].drop_duplicates()
udrugs_gsnvc = cat_pres.gsn[cat_pres.A<0].value_counts()
gsn_udrugs = udrugs_gsnvc[udrugs_gsnvc>1].index.values

In [ ]:
cat.append('gsn')
cat_gsn = cat_pres.loc[cat_pres.gsn.isin(gsn_udrugs),cat].groupby('gsn').max()
cat_gsn = cat_gsn[cat_gsn.A>-1]
cat_gsn

In [ ]:
cat = cat[:-1]
cat_pres[cat] = cat_pres[cat].where(cat_pres[cat]>=0)

In [ ]:
cat_pres.set_index(ind,append = True)

In [ ]:
cat_pres.set_index('gsn', inplace = True)
cat_pres.update(cat_gsn)
cat_pres.set_index(ind)

In [ ]:
cat_pres.head()

In [ ]:
sum(cat_pres.drug.str.contains('polyvalent'))

In [ ]:
cat_pres.gsn[misspelt_drugs_bool].value_counts().describe()

In [ ]:
(5303415+7791711)/len(cat_pres)

In [ ]:
cat

In [ ]:
cat = list('ABCDGHJLMNPRSV')
cat.append('drug')
cat_drug = cat_pres[cat].fillna(0.0001).groupby('formulary_drug_cd',dropna = False).sum()
cat_drug

In [ ]:
atr4_pres.atc4_name.nunique()

The missing values labelled as 0 are now empty strings due to the conversion. There may be other values that are not 0, maybe even some could be in other ndc formats (hyphenated or with less digits). We can find out how many rows have which number of digits.

So all fields that do not have 9-digit numbers have no digits. We can consider all of these as missing values.

More than 12% of ndc fields are missing. Some of the drugs listed here may not have an ndc (Sodium Chloride flush on 2nd row for instance), but others may be mistakes. NDC is specific to product, not to drug (same drug from different companies will have a different code) so filling from the table would not be very accurate. Given that what we need the NDC just to classify our drugs, we will leave these fields blank and will fill the categories instead after NDC-matching.


In [ ]:
prescriptions.ndc[~missing_ndc].drop_duplicates().to_csv('Unique_NDCs.csv')

Let us see if we can fill these missing values by using the other fields in the table. pharmacy_id, gsn, drug and formulary_drug_cd could be used for this.

In [ ]:
prescriptions.formulary_drug_cd.info()

In [ ]:
fdrugna = prescriptions[prescriptions.formulary_drug_cd.isna()]

In [ ]:
def df_filler(x):
    

prescriptions.formulary_drug_cd[prescriptions.drug.isin(fdrugna.drug)].isna()

In [ ]:
prescriptions.gsn.value_counts()

In [ ]:
prescriptions.pharmacy_id.value_counts()

In [ ]:
prescriptions.formulary_drug_cd.value_counts()

In [ ]:
prescriptions.formulary_drug_cd.info()

In [ ]:
prescriptions.drug = prescriptions.drug.str.lower()
prescriptions.drug.value_counts().describe()

In [ ]:
ndc_bool = prescriptions.ndc!=u'' & prescriptions 
print(sum(ndc_bool))
print(sum(~ndc_bool))

In [ ]:
elswhr_bool = prescriptions.loc[~ndc_bool,'drug'].isin(prescriptions.loc[ndc_bool,'drug'])
print(round(sum(elswhr_bool)/sum(~ndc_bool)*100,2))

In [ ]:
prescriptions.loc[elswhr_bool,'ndc'] = prescriptions.loc[ndc_bool*prescriptions.,'ndc']

In [ ]:
prescriptions.ndc.nunique()

In [ ]:
import numpy as np

In [ ]:
medication = pd.read_json(r'C:\Users\Nacho\Downloads\Medication.ndjson', lines = True)

In [ ]:
medication.shape

In [ ]:
prescriptions.ndc.head()

In [ ]:
ndc_desc = pd.read_csv("G:\My Drive\Brainstation\Capstone project\product.csv",usecols = ['PRODUCTNDC','NONPROPRIETARYNAME','PHARM_CLASSES'])

In [ ]:
ndc_desc = ndc_desc.loc[~ndc_desc.PHARM_CLASSES.isna()].reset_index(drop = True)

In [ ]:
ndc_desc.head()

In [ ]:
def sht2nineNDC(x):
    x = x.split('-')
    x = x[0].zfill(5) + x[1].zfill(4)
    
    return x
    

ndc_desc.PRODUCTNDC = ndc_desc.PRODUCTNDC.apply(sht2nineNDC)

In [ ]:
ndc_desc.PRODUCTNDC.head()

In [ ]:
classpername = ndc_desc[['NONPROPRIETARYNAME','PHARM_CLASSES']].groupby('NONPROPRIETARYNAME').nunique()

In [ ]:
classperndc = ndc_desc[['PRODUCTNDC','PHARM_CLASSES']].groupby('PRODUCTNDC').nunique()

In [ ]:
classpername.value_counts()

In [ ]:
classperndc.value_counts()

In [ ]:
np.mean(prescriptions.ndc.isna())*100

In [ ]:
prescriptions.loc[prescriptions.ndc.isna()].head(10)

In [ ]:
ndc_desc.rename(columns = {'PRODUCTNDC' : 'ndc'},inplace = True)

In [ ]:
presfinal = pd.merge(prescriptions,ndc_desc, how = 'left',on = 'ndc')

In [ ]:
np.mean(presfinal.PHARM_CLASSES.isna())

In [ ]:
np.mean(presfinal.NONPROPRIETARYNAME.isna())

In [ ]:
presfinal[['ndc','drug','NONPROPRIETARYNAME','PHARM_CLASSES']]

In [ ]:
presfinal.hadm_id.nunique()

In [ ]:
len(presfinal.PHARM_CLASSES.isna())-sum(presfinal.PHARM_CLASSES.isna())

In [ ]:
classperhadm = presfinal[['PHARM_CLASSES','hadm_id']].groupby('hadm_id').nunique()
classperhadm.describe()

In [ ]:
classperhadm.loc[classperhadm.PHARM_CLASSES == 0].count()

In [ ]:
prescriptions.ndc.apply(lambda x: str(x)[:-2].zfill(9))

In [ ]:
print(ndc_desc.NONPROPRIETARYNAME.nunique())
print(ndc_desc_clean.NONPROPRIETARYNAME.nunique())

In [ ]:
ndc_desc.shape

In [ ]:
ndc_desc.PRODUCTNDC.apply(lambda x: int(x.replace('-','')))

In [ ]:
ndc_desc.PHARM_CLASSES.value_counts

In [ ]:
poe = pd.read_csv('MIMIC IV ED/poe.csv')

In [ ]:
poe.order_status.value_counts(normalize = True)

In [ ]:
poe.loc[poe.order_type == 'Lab'].order_subtype.value_counts()

In [ ]:
poe.order_type.value_counts()

In [ ]:
admissions = pd.read_csv('MIMIC IV ED/admissions.csv')

In [ ]:
admissions.race.value_counts(normalize = True)

In [ ]:
sum(admissions.admission_location.isna())

In [ ]:
admissions.admission_location.value_counts(normalize = True)

In [ ]:
transfers = pd.read_csv('MIMIC IV ED/transfers.csv')

In [ ]:
sum(transfers.eventtype == 'discharge')

In [ ]:
transfers.careunit.value_counts(normalize = True)

In [ ]:
transfers.careunit.unique()

In [ ]:
transfers[['intime','outtime']] = pd.to_datetime(transfers[['intime', 'outtime']])

In [ ]:
transfers.info(show_counts = True)

In [ ]:
bool_outs

In [ ]:
transfers.describe()

In [ ]:
bool_discharge = transfers.eventtype=='discharge'
bool_carenan = transfers.careunit.isna()
bool_outtime = transfers.outtime.isna()

In [ ]:
print(all(bool_discharge == bool_carenan))
print(all(bool_discharge == bool_outtime))


In [ ]:
transfers.head()

In [ ]:
services = pd.read_csv('MIMIC IV ED/services.csv')

In [ ]:
services.shape

In [ ]:
services.head()

In [ ]:
services.prev_service.value_counts()

In [ ]:
transfers.eventtype.value_counts()

In [ ]:
transfers.

In [ ]:
emar = pd.read_csv('MIMIC IV/emar.csv')

In [ ]:
emar.medication.nunique()

In [ ]:
emar.head()

In [ ]:
transfers.careunit.value_counts()

In [ ]:
print('Number of patients:',transfers['subject_id'].nunique())
print('Number of transfers (and rows number):',transfers['transfer_id'].nunique())
print('Number of stays:',transfers['hadm_id'].nunique())
print('Number of departments:',transfers['careunit'].nunique())

In [ ]:
sum(transfers['careunit'].isna())

In [ ]:
print('Number of departments:',transfers['careunit'].unique())

In [ ]:
admissions = pd.read_csv('MIMIC IV ED/admissions.csv')

In [ ]:
pd.to_datetime(admissions.admittime,infer_datetime_format = True).describe(datetime_is_numeric = True)

In [ ]:
admissions.discharge_location.unique()

In [ ]:
admissions.head()

In [ ]:
len(admissions)

In [ ]:
print('Number of patients:',admissions['subject_id'].nunique())
print('Number of transfers (and length of rows):',admissions['transfer_id'].nunique())
print('Number of stays:',transfers['hadm_id'].nunique())
print('Number of departments:',transfers['careunit'].nunique())

In [ ]:
edstays['disposition'].value_counts()

In [ ]:
diagnosis.icd_version.value_counts()

In [ ]:
diagnosis['icd_title'].value_counts().head()

In [ ]:
print('\033[1m' + 'diagnosis')
display(diagnosis.head())
display(diagnosis.info())
print('\033[1m' + 'edstays')
display(edstays.head())
display(diagnosis.info())
print('\033[1m' + 'medrecon')
display(medrecon.head())
display(medrecon.info())
print('\033[1m' + 'pyxis')
display(pyxis.head())
display(pyxis.info())
print('\033[1m' + 'triage')
display(triage.head())
display(triage.info())
print('\033[1m' + 'vitalsign')
display(vitalsign.head())
display(vitalsign.info())